# Projeto Final - Análise de Gols em uma Partida de Futebol
Autor: Paulo Victor Lima |   Orientador : Sergio Lima Netto |    Universidade Federal do Rio de Janeiro - Escola Politécnica - Departamente de Engenharia Eletrônica e de Computação

## Importação das Bibliotecas

In [ ]:
#Instalação das bibliotecas

!pip install pandas
!pip install numpy
!pip install seaborn
!pip install matplotlib.pyplot
!pip install plotly.express
!pip install glob

In [3]:
#Importação das bibliotecas

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import glob
import os

## Importação Base de Dados

In [21]:
from numpy import loadtxt

features_files_type = []
features_files_type = loadtxt("..\config\Features_Types.dat", comments="#", delimiter='\n', dtype=str)

print("All Features : ")
print(features_files_type)
print("\n ###### Begin reading data ######")

moments_data_frame_array = []
for feature_type in features_files_type:
    pathName = '..\\database\\**\\' + feature_type
    print(pathName)
    all_files_by_feature = glob.glob(pathName, recursive = True)

    feature_data_frame_array = []
    for file in all_files_by_feature:
        feature_data_frame = pd.read_csv(file, index_col=False, sep =',', thousands=r".", header=None)
        feature_data_frame['file '+feature_type] = file.split('/')[-1]
        feature_data_frame_array.append(feature_data_frame)

    feature_data_frame = pd.concat(feature_data_frame_array, ignore_index=True, sort=True)
    moments_data_frame_array.append(feature_data_frame)

moments_data_frame = pd.concat(moments_data_frame_array, ignore_index=True, sort=True)
moments_data_frame.to_csv("moments.csv", index=False, sep=';')


        
##########################################################################################################################
    

# all_files_by_feature = glob.glob('..\database\**\dc_hue_mean.csv', recursive = True)

# all_df = []
# for f in all_files_by_feature:
#     df = pd.read_csv(f, index_col=False, sep =',', thousands=r".", header=None)
#     df['file'] = f.split('/')[-1]
#     all_df.append(df)
    
# merged_df = pd.concat(all_df, ignore_index=True, sort=True)
# merged_df.to_csv( "merged.csv", index=False)


All Features : 
['dc_hue_mean.csv' 'dc_percent.csv' 'em_cs_energy.csv'
 'em_cs_energy_diff.csv']

 ###### Begin reading data ######
..\database\**\dc_hue_mean.csv


TypeError: list indices must be integers or slices, not builtin_function_or_method